In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # Use the first GPU
    print("using GPU")
else:
    device = torch.device("cpu")  # Use CPU if no GPU is available
    print("using CPU")

using GPU


In [2]:
# compile all reddit posts together

import pandas as pd
import json
import tqdm

datapath = "data/userReddit.csv"
reddit_data = pd.read_csv(datapath)

In [14]:
reddit_data.head()

,content,userId,createdTime,sentiment,preprocessed_content
0,Hi I'm Rick Astley. Good to be back here again...,ReallyRickAstley,1.697728e+09,"{'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'comp...",hi im rick astley good back my new album are w...
1,Live stream chat with my new album and upcomin...,ReallyRickAstley,1.697647e+09,"{'neg': 0.06, 'neu': 0.713, 'pos': 0.227, 'com...",live stream chat new album upcoming ama hi im ...
2,Coming soon…\n,ReallyRickAstley,1.697149e+09,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",coming soon
3,Rick Astley (me) - Never Gonna Stop [Pop]\n,ReallyRickAstley,1.693502e+09,"{'neg': 0.0, 'neu': 0.726, 'pos': 0.274, 'comp...",rick astley never gonna stop pop
4,Rick Astley (me) - Dippin My Feet [Pop]\n,ReallyRickAstley,1.689269e+09,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",rick astley dippin my feet pop


In [10]:
# for user in tqdm(users, desc="Processing user posts", unit="user"):

f = reddit_data['userId'].value_counts()
f[:10],f[10:20], f[20:30]

(userId
 CesaroSalad             791
 serventofgaben          707
 thecrazydeviant         445
 schweenieboy            335
 angelicdemon13          244
 paspa1801               207
 SomeGuyInDeutschland    204
 Myamoxomis              186
 parisianseine           133
 Yotroller02             120
 Name: count, dtype: int64,
 userId
 suryakantamangaraj    112
 throwawaynumber53     111
 ams712                105
 Afol456                95
 GenTryxx               79
 PsychoPotency          78
 Ice-Senpai0323         76
 ReallyRickAstley       71
 gxzzymo                69
 collect_Info_1960      68
 Name: count, dtype: int64,
 userId
 cookiecutie707      67
 e-vanilla           66
 ShakinAssFaJeeps    63
 thesomeonetokno     60
 noodlepooodle       59
 milkystarrgirl      56
 Itsdon_on           52
 gr0wyourhair        51
 wd26                47
 flatbootybih        36
 Name: count, dtype: int64)

In [12]:
user = "gr0wyourhair"

user_history = reddit_data[reddit_data['userId'] == user]
sorted_history = user_history.sort_values(by='createdTime')

In [19]:
type(sorted_history['preprocessed_content'])

pandas.core.series.Series

In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import SequenceClassificationExplainer


2023-11-17 19:36:34.338652: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 19:36:34.954319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [24]:
model_name = "SamLowe/roberta-base-go_emotions"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [25]:
def sequenceclassification(model, tokenizer, sentence):
    cls_explainer = SequenceClassificationExplainer(model, tokenizer)
    word_attributions = cls_explainer(sentence)
    label = cls_explainer.predicted_class_name
    return word_attributions, label

In [26]:
compiled_word_attributions = []
compiled_labels = []

for post in sorted_history['preprocessed_content']:
    word_attributions, label = sequenceclassification(model, tokenizer, post)
    compiled_word_attributions.append(word_attributions)
    compiled_labels.append(label)

In [85]:
# ! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [86]:
#Define a function to perform sentiment analysis using VADER
def get_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment

In [90]:
# getting sentiment
analyzer = SentimentIntensityAnalyzer()
sorted_history['sentiment'] = sorted_history['preprocessed_content'].apply(get_sentiment)
sentiment = sorted_history['sentiment']

In [37]:
timestamps = sorted_history['createdTime']
# timestamps

In [72]:
from datetime import datetime, timezone

def get_time(timestamp):
    dt_utc = datetime.utcfromtimestamp(timestamp).replace(tzinfo=timezone.utc)
    
    # Extract year, month, day, hour, minute, and second
    year = dt_utc.year
    month = dt_utc.month
    day = dt_utc.day
    hour = dt_utc.hour
    mint = dt_utc.minute
    second = dt_utc.second
    
    return year, month, day, hour, mint, sec
    

In [80]:
# top_10_words_score = []
# top_10_words = []
all_post_top_words = []

for wa in compiled_word_attributions:
    sorted_wa = sorted(wa, key=lambda x: x[1], reverse=True)[:10]
    top_words = []
    for word, score in sorted_wa:
        if score>0:
            top_words.append((word, score))
    all_post_top_words.append(top_words)
    # top_10_words_score.append(sorted_wa)
    # top_10 = [worddict[0] for worddict in sorted_wa]
    # top_10_words.append(top_10)


In [82]:
# all_post_top_words

In [92]:
# Open a file in write mode ('w')
with open('output.txt', 'w') as file:
    for timestamp, words, s, label in zip(timestamps, all_post_top_words, sentiment, compiled_labels):
        year, month, day, hour, mint, sec = get_time(timestamp)
        if hour > 12:
            hour -= hour - 12
        # Redirect print statements to the file
        print(f"Date: {year}/{month}/{day}; Time: {hour}:{mint}:{sec}", file=file)
        print("Top words used: ", file=file)
        for word, score in words:
            print(word, " : ", score, file=file)
        print(f"Sentiment: {s}", file=file)
        print(f"Classified emotion: {label}", file=file)
        print("", file=file)

print('Output has been written to output.txt')


Output has been written to output.txt
